In [ ]:
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
url_base = 'https://www.beermile.com/display/'
max_event_id = 151381

In [ ]:
%%time
# Look for all events by numerical ID
for i in range(2, max_event_id):
    # Website uses sequential 
    event = f'event_{i}'
    url = f'{url_base}{event}'
    result = requests.get(url)
    print('Status Code: {result.status_code}')
    print(f'URL: {result.url}')
    content = result.content
    print()
    
print('Done')